In [7]:
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.preprocessing import LabelEncoder

In [8]:
df = pd.read_csv("/Users/zhangxijing/MasterNEU/INFO6105DataScienceEngineeringMethodsandTools/Dataset/DS_Dataset.csv")
df_copy = df.copy()
print(df_copy)

       Isolate_No  Patient_Age                 Bacteria Antimicrobial  \
0      2010V-1004         21.0     Vibrio alginolyticus          AMP    
1      2010V-1004         21.0     Vibrio alginolyticus          CIP    
2      2010V-1004         21.0     Vibrio alginolyticus          COT    
3      2010V-1004         21.0     Vibrio alginolyticus          TET    
4      2010V-1005         22.0             Vibrio other          AMP    
...           ...          ...                      ...           ...   
2326  2013AW-0105         30.0  Vibrio parahaemolyticus          CIP    
2327  2013AW-0105         30.0  Vibrio parahaemolyticus          COT    
2328  2013AW-0105         30.0  Vibrio parahaemolyticus          CTX    
2329  2013AW-0105         30.0  Vibrio parahaemolyticus          GEN    
2330  2013AW-0105         30.0  Vibrio parahaemolyticus          IMI    

          MIC MIC_Interpretation  
0     128.000          resistant  
1       0.125        susceptible  
2       0.064     

In [9]:
# delete first column
df_copy = df_copy.drop(df_copy.columns[0], axis=1)

#remove duplicates
df_copy = df_copy.drop_duplicates()
print(df_copy)

      Patient_Age                 Bacteria Antimicrobial      MIC  \
0            21.0     Vibrio alginolyticus          AMP   128.000   
1            21.0     Vibrio alginolyticus          CIP     0.125   
2            21.0     Vibrio alginolyticus          COT     0.064   
3            21.0     Vibrio alginolyticus          TET     1.000   
4            22.0             Vibrio other          AMP     3.000   
...           ...                      ...           ...      ...   
2326         30.0  Vibrio parahaemolyticus          CIP     0.190   
2327         30.0  Vibrio parahaemolyticus          COT     0.190   
2328         30.0  Vibrio parahaemolyticus          CTX     0.190   
2329         30.0  Vibrio parahaemolyticus          GEN     0.750   
2330         30.0  Vibrio parahaemolyticus          IMI     0.190   

     MIC_Interpretation  
0             resistant  
1           susceptible  
2           susceptible  
3           susceptible  
4                   NaN  
...            

In [10]:
# convert columns to integers
dataframe = df_copy.copy()
dataframe['Bacteria'] = LabelEncoder().fit_transform(dataframe['Bacteria'])
dataframe['Antimicrobial'] = LabelEncoder().fit_transform(dataframe['Antimicrobial'])
dataframe['MIC_Interpretation'] = LabelEncoder().fit_transform(dataframe['MIC_Interpretation'])

dataframe.loc[dataframe['MIC_Interpretation'] == 3, 'MIC_Interpretation'] = "NaN"

# Print the DataFrame
print(dataframe)
dataframe.to_csv("HW1_1_Result222.csv")

      Patient_Age  Bacteria  Antimicrobial      MIC MIC_Interpretation
0            21.0         0              0  128.000                  1
1            21.0         0              2    0.125                  2
2            21.0         0              3    0.064                  2
3            21.0         0              7    1.000                  2
4            22.0         7              0    3.000                NaN
...           ...       ...            ...      ...                ...
2326         30.0         8              2    0.190                  2
2327         30.0         8              3    0.190                  2
2328         30.0         8              4    0.190                NaN
2329         30.0         8              5    0.750                  2
2330         30.0         8              6    0.190                  2

[1722 rows x 5 columns]


In [710]:
##MICE
imputer = IterativeImputer(max_iter=10, random_state=0)
imputed_dataset = imputer.fit_transform(dataframe)
imputed_dataframe = pd.DataFrame(imputed_dataset, columns=dataframe.columns)

# find the closest integer
imputed_dataframe['MIC_Interpretation'] = imputed_dataframe['MIC_Interpretation'].round()
print(imputed_dataframe)

      Patient_Age  Bacteria  Antimicrobial      MIC  MIC_Interpretation
0            21.0       0.0            0.0  128.000                 1.0
1            21.0       0.0            2.0    0.125                 2.0
2            21.0       0.0            3.0    0.064                 2.0
3            21.0       0.0            7.0    1.000                 2.0
4            22.0       7.0            0.0    3.000                 2.0
...           ...       ...            ...      ...                 ...
1717         30.0       8.0            2.0    0.190                 2.0
1718         30.0       8.0            3.0    0.190                 2.0
1719         30.0       8.0            4.0    0.190                 2.0
1720         30.0       8.0            5.0    0.750                 2.0
1721         30.0       8.0            6.0    0.190                 2.0

[1722 rows x 5 columns]


In [711]:
##Binning
imputed_dataframe['MIC_bin'] = pd.qcut(imputed_dataframe['MIC'], q=3)
imputed_dataframe['MIC'] = pd.Series ([interval.mid for interval in imputed_dataframe['MIC_bin']])

print(imputed_dataframe)

      Patient_Age  Bacteria  Antimicrobial       MIC  MIC_Interpretation  \
0            21.0       0.0            0.0  128.5000                 1.0   
1            21.0       0.0            2.0    0.0630                 2.0   
2            21.0       0.0            3.0    0.0630                 2.0   
3            21.0       0.0            7.0    0.5625                 2.0   
4            22.0       7.0            0.0  128.5000                 2.0   
...           ...       ...            ...       ...                 ...   
1717         30.0       8.0            2.0    0.5625                 2.0   
1718         30.0       8.0            3.0    0.5625                 2.0   
1719         30.0       8.0            4.0    0.5625                 2.0   
1720         30.0       8.0            5.0    0.5625                 2.0   
1721         30.0       8.0            6.0    0.5625                 2.0   

             MIC_bin  
0       (1.0, 256.0]  
1     (0.001, 0.125]  
2     (0.001, 0.12

In [712]:
##Detect Outliers
q3, q1 = np.percentile(imputed_dataframe['MIC'], [75 ,25])
fence = 1.5 * (q3 - q1)
upper_band = q3 + fence
lower_band = q1 - fence
print( 'q1=',q1,' q3=', q3, ' IQR=', q3-q1, ' upper=', upper_band, 'lower=',lower_band)
outliers = [val for val in imputed_dataframe['MIC'] if val < lower_band or val > upper_band]
print('Outliers=', outliers)


q3, q1 = np.percentile(imputed_dataframe['Patient_Age'], [75 ,25])
fence = 1.5 * (q3 - q1)
upper_band = q3 + fence
lower_band = q1 - fence
print( 'q1=',q1,' q3=', q3, ' IQR=', q3-q1, ' upper=', upper_band, 'lower=',lower_band)
outliers = [val for val in imputed_dataframe['Patient_Age'] if val < lower_band or val > upper_band]
print('Outliers=', outliers)

# Create a mask to identify rows with outliers
outliers_mask = (imputed_dataframe['Patient_Age'] < lower_band) | (imputed_dataframe['Patient_Age'] > upper_band)

# Remove rows with outliers
filtered_dataframe = imputed_dataframe[~outliers_mask]

print('DataFrame without outliers:')
print(filtered_dataframe)


# Create a mask to identify rows with outliers in df_copy
outliers_mask_copy = (df_copy['Patient_Age'] < lower_band) | (df_copy['Patient_Age'] > upper_band)

# Remove rows with outliers from df_copy
filtered_df_copy = df_copy[~outliers_mask_copy]

# Print or handle the DataFrame without outliers in df_copy
print('df_copy without outliers:')
print(filtered_df_copy)

q1= 0.063  q3= 128.5  IQR= 128.437  upper= 321.1555 lower= -192.59250000000003
Outliers= []
q1= 26.0  q3= 63.0  IQR= 37.0  upper= 118.5 lower= -29.5
Outliers= [145.0, 145.0, 145.0, 145.0, -34.0, -34.0, -34.0, -34.0, 154.0, 733.0, -45.0, -45.0, -45.0, -45.0]
DataFrame without outliers:
      Patient_Age  Bacteria  Antimicrobial       MIC  MIC_Interpretation  \
0            21.0       0.0            0.0  128.5000                 1.0   
1            21.0       0.0            2.0    0.0630                 2.0   
2            21.0       0.0            3.0    0.0630                 2.0   
3            21.0       0.0            7.0    0.5625                 2.0   
4            22.0       7.0            0.0  128.5000                 2.0   
...           ...       ...            ...       ...                 ...   
1717         30.0       8.0            2.0    0.5625                 2.0   
1718         30.0       8.0            3.0    0.5625                 2.0   
1719         30.0       8.0   

In [713]:
## Normalize
from sklearn.preprocessing import StandardScaler

# Extract numeric columns (assuming 'MIC' and 'Patient_Age' are numeric columns)
numeric_columns = ['MIC', 'Patient_Age']

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the selected columns using .loc
filtered_dataframe.loc[:, numeric_columns] = scaler.fit_transform(filtered_dataframe.loc[:, numeric_columns])

print(filtered_dataframe)

      Patient_Age  Bacteria  Antimicrobial       MIC  MIC_Interpretation  \
0       -1.057835       0.0            0.0  1.445193                 1.0   
1       -1.057835       0.0            2.0 -0.696166                 2.0   
2       -1.057835       0.0            3.0 -0.696166                 2.0   
3       -1.057835       0.0            7.0 -0.687838                 2.0   
4       -1.012737       7.0            0.0  1.445193                 2.0   
...           ...       ...            ...       ...                 ...   
1717    -0.651953       8.0            2.0 -0.687838                 2.0   
1718    -0.651953       8.0            3.0 -0.687838                 2.0   
1719    -0.651953       8.0            4.0 -0.687838                 2.0   
1720    -0.651953       8.0            5.0 -0.687838                 2.0   
1721    -0.651953       8.0            6.0 -0.687838                 2.0   

             MIC_bin  
0       (1.0, 256.0]  
1     (0.001, 0.125]  
2     (0.001, 0.12

/var/folders/4x/93t8x08s0s15_93t7b53wtzr0000gn/T/ipykernel_10121/1865993892.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dataframe.loc[:, numeric_columns] = scaler.fit_transform(filtered_dataframe.loc[:, numeric_columns])


In [714]:
filtered_dataframe.reset_index(drop=True, inplace=True)
filtered_dataframe.index = filtered_dataframe.index + 1

filtered_df_copy.reset_index(drop=True, inplace=True)
filtered_df_copy.index = filtered_df_copy.index + 1


# send to result to df_copy
filtered_df_copy['Patient_Age'] = filtered_dataframe['Patient_Age']
filtered_df_copy['MIC'] = filtered_dataframe['MIC']
filtered_df_copy['MIC_Interpretation'] = filtered_dataframe['MIC_Interpretation']

print(filtered_df_copy)
print(filtered_df_copy.shape)

# get the result and create the file
filtered_df_copy.to_csv("HW1_1_Result.csv")


      Patient_Age                 Bacteria Antimicrobial       MIC  \
1       -1.057835     Vibrio alginolyticus          AMP   1.445193   
2       -1.057835     Vibrio alginolyticus          CIP  -0.696166   
3       -1.057835     Vibrio alginolyticus          COT  -0.696166   
4       -1.057835     Vibrio alginolyticus          TET  -0.687838   
5       -1.012737             Vibrio other          AMP   1.445193   
...           ...                      ...           ...       ...   
1704    -0.651953  Vibrio parahaemolyticus          CIP  -0.687838   
1705    -0.651953  Vibrio parahaemolyticus          COT  -0.687838   
1706    -0.651953  Vibrio parahaemolyticus          CTX  -0.687838   
1707    -0.651953  Vibrio parahaemolyticus          GEN  -0.687838   
1708    -0.651953  Vibrio parahaemolyticus          IMI  -0.687838   

      MIC_Interpretation  
1                    1.0  
2                    2.0  
3                    2.0  
4                    2.0  
5                    2.

/var/folders/4x/93t8x08s0s15_93t7b53wtzr0000gn/T/ipykernel_10121/2944302322.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_copy['Patient_Age'] = filtered_dataframe['Patient_Age']
/var/folders/4x/93t8x08s0s15_93t7b53wtzr0000gn/T/ipykernel_10121/2944302322.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_copy['MIC'] = filtered_dataframe['MIC']
/var/folders/4x/93t8x08s0s15_93t7b53wtzr0000gn/T/ipykernel_10121/2944302322.py:13: SettingWithCopyWarning: 
A value is trying to be se